### Authorship
@author: Alexandre Pereira Santos <br>
alexandre.santos(at)lmu.de<br>
- many scripts in this notebook com from geemap (see below)

### Tasks
- get Global Human Settlement Layer population estimates data using the Google Earth Engine functionalities
- clip it to an AOI

### Prerequisites
- earth engine api (ee) https://developers.google.com/earth-engine/apidocs
- gee map (gee), https://geemap.org/ by Qiusheng Wu
- create an account and a Google Cloud project in Google Developer, see here: https://developers.google.com/earth-engine/guides/auth

# init

In [1]:
import ee
import geemap
from pathlib import Path
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os

# extract coordinates from the bounding box
def get_coords(gdf):
    coords = gdf.to_crs(epsg='4326').envelope
    epsg_coords = coords.crs.to_epsg()
    transform = [coords.bounds.values[0][0], coords.bounds.values[0][1], coords.bounds.values[0][2], coords.bounds.values[0][3]]
    roi = ee.Geometry.BBox(west=transform[0], south=transform[1], east=transform[2], north=transform[3])
    rec_roi = ee.Geometry.Rectangle(transform[0],transform[1],transform[2],transform[3])
    coi = roi.centroid(maxError=1)
    return epsg_coords, roi, rec_roi, coi

# define a function that exports the image to Google Drive
def gee_to_tiff(input_layer, out_folder, output_filename, roi):
    projection = input_layer.projection().getInfo()
    
    try:
        if not os.path.exists(out_folder):
            os.mkdir(out_folder)
        
        # Export a GEE layer to raster Gtiff
        geemap.ee_export_image_to_drive(
            input_layer,
            description=output_filename,
            folder=out_folder,
            maxPixels=1e13, #ressamples to a low resolution
            region = roi,
            crs = projection['crs'],
            fileFormat = 'GeoTIFF',
            scale = 10           
        )
        #raster = 
        #return raster
    except Exception as e:
        print("An error has occurred:", e)

# 

In [19]:
# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-alexandresantosgeographie') # Replace with your GEE project

print(ee.__version__)

0.1.406


In [7]:
# check if the authentication is working
print(ee.Image("JRC/GHSL/P2023A/GHS_POP/1975").get("smod_code").getInfo())

None


# imports

In [20]:
#input a vector and a raster file
AOI_path = Path('../data/processed/')

AOI_file = 'LIM_rs_floods_2024_bbox_basins_A.shp' # AOI for the affected area, includes the watersheds Taquari, Alto e Baixo Jacui, Pardo, Vacacaí, Gravatai, Sinos, Caí, and Guaíba

mun_path = Path('../data/external/')
mun_file = 'LIM_RS_municipalites_2022_A.shp' # municipalities in RS

#read the vector files
AOI_gdf = gpd.read_file(AOI_path/AOI_file).to_crs(epsg=4326)
mun_gdf = gpd.read_file(mun_path/mun_file).to_crs(epsg=4326)

aoi_coords, aoi_roi, aoi_rec_roi, aoi_coi = get_coords(AOI_gdf)
mun_coords, mun_roi, mun_rec_roi, mun_coi = get_coords(mun_gdf)

# get the coordinates
print(AOI_gdf.crs, mun_gdf.crs)

EPSG:4326 EPSG:4326


In [21]:
# methods from https://www.geo.fu-berlin.de/en/v/geo-it/gee/2-monitoring-ndvi-nbr/2-2-calculating-indices/ndvi-s2/index.html

def roi_ndvi(aoi_gdf):
    aoi_coords = aoi_gdf.to_crs(epsg='4326').envelope
    #print('N',aoi_coords.bounds.values[0][3],'S',aoi_coords.bounds.values[0][1],'W',aoi_coords.bounds.values[0][0],'E',aoi_coords.bounds.values[0][2])
    transform = [aoi_coords.bounds.values[0][0], aoi_coords.bounds.values[0][1], aoi_coords.bounds.values[0][2], aoi_coords.bounds.values[0][3]]
    roi = ee.Geometry.BBox(west=transform[0], south=transform[1], east=transform[2], north=transform[3])
    
    s2=ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterBounds(roi).filterDate('2023-01-01', '2024-05-30').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 7)).median()
    # Use median to reduce noise and clouds

    # Calculate NDVI
    ndvi = s2.normalizedDifference(['B8', 'B4']).rename('NDVI')  

    return ndvi

In [23]:
aoi_ndvi = roi_ndvi(AOI_gdf)

# Define visualization parameters
ndviParams = {min: 0,max: 1,'palette': ['blue', 'white', 'green']}

Map = geemap.Map()
# Visualize the NDVI
Map.centerObject(aoi_roi, 8)
Map.addLayer(aoi_ndvi.clip(aoi_roi), ndviParams, 'NDVI')
Map

Map(center=[-29.473686646032423, -52.23468725999999], controls=(WidgetControl(options=['position', 'transparen…

In [9]:
aoi_coords = AOI_gdf_JAK.to_crs(epsg='4326').envelope
#print('N',aoi_coords.bounds.values[0][3],'S',aoi_coords.bounds.values[0][1],'W',aoi_coords.bounds.values[0][0],'E',aoi_coords.bounds.values[0][2])
transform = [aoi_coords.bounds.values[0][0], aoi_coords.bounds.values[0][1], aoi_coords.bounds.values[0][2], aoi_coords.bounds.values[0][3]]
roi = ee.Geometry.BBox(west=transform[0], south=transform[1], east=transform[2], north=transform[3])
rec_roi = ee.Geometry.Rectangle(transform[0],transform[1],transform[2],transform[3])
coi = roi.centroid(maxError=1)

In [18]:
# defining EE collection
# Sentinel-2 L2A Harmonized collection

# code based on https://developers.google.com/earth-engine/datasets/catalog/JRC_GHSL_P2023A_GHS_POP

Map = geemap.Map()
Map.centerObject(coi, zoom=9)

s2=ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterBounds(roi).filterDate('2023-01-01', '2024-05-30').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 7)).median()
# Use median to reduce noise and clouds

# Calculate NDVI
ndvi = s2.normalizedDifference(['B8', 'B4']).rename('NDVI')

# Define visualization parameters
ndviParams = {min: 0,max: 1,'palette': ['blue', 'white', 'green']}

# Visualize the NDVI
Map.centerObject(roi, 10)
Map.addLayer(ndvi.clip(roi), ndviParams, 'NDVI')
#Map.addLayer(roi, {'color': 'red'}, 'AOI')
Map

Map(center=[-6.356093939017252, 106.7482247859414], controls=(WidgetControl(options=['position', 'transparent_…

In [19]:
projection = JAK_coi.projection().getInfo()
print(projection)

{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}


In [17]:
#aoi_clip = aoi_ndvi.clip(aoi_rec_roi).unmask()
export = gee_to_tiff(aoi_ndvi, 'data', 'RSFL24_ENV_NDVI_2023_AOI', aoi_rec_roi)